# BÀI TẬP THỰC HÀNH 1
## Thực hiện thống kê mô tả trên tập dữ liệu về phân loại chất lượng rượu đỏ
Nguồn dữ liệu: [Red Wine Quality Dataset - Kaggle](https://www.kaggle.com/code/eisgandar/red-wine-quality-eda-classification)


## 1. Nhập thư viện

In [52]:
import pandas as pd
import numpy as np
from scipy import stats

## 2. Đọc dữ liệu
Đọc tập dữ liệu winequality-red.csv vào DataFrame wine.

In [53]:
wine = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/EDA/winequality-red.csv", sep=",")

## 3. Thông tin tổng quan



In [54]:
# shape
print(f'+ Shape: {wine.shape}')
# types
print(f'+ Data Types: \n{wine.dtypes}')
# head, tail
print(f'+ Contents: ')
display(wine.head(5))
display(wine.tail(5))
# info
wine.info()
desc = wine.describe().T


+ Shape: (1599, 12)
+ Data Types: 
fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object
+ Contents: 


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
1598,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


Các biến có quy mô giá trị khác nhau, ví dụ residual sugar và density dao động khác biệt. alcohol có giá trị trung bình khoảng 10.4%, điều này ảnh hưởng mạnh đến chất lượng rượu.

## 4. Kiểm tra dữ liệu thiếu / giá trị bất hợp lý

In [55]:
missing = wine.isnull().sum()
n_duplicated = wine.duplicated().sum()

missing_percent = (missing / len(wine)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing Percent': missing_percent
})
display(missing_df)
print(f'+ Số dòng trùng: {n_duplicated}')

,Missing Count,Missing Percent
fixed acidity,0,0.0
volatile acidity,0,0.0
citric acid,0,0.0
residual sugar,0,0.0
chlorides,0,0.0
free sulfur dioxide,0,0.0
total sulfur dioxide,0,0.0
density,0,0.0
pH,0,0.0
sulphates,0,0.0


+ Số dòng trùng: 240


Tập dữ liệu gồm 1359 dòng và 12 cột — mỗi dòng đại diện cho một mẫu rượu, các cột là các chỉ số hóa học hoặc chất lượng cảm quan. Không có dữ liệu bị thiếu, tuy nhiên có vài cột có giá trị 0 (như citric acid hoặc residual sugar), cần cân nhắc khi phân tích.

## 5. Xử lý dữ liệu thiếu

In [56]:
numeric_cols = wine.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    median_val = wine[col].median()
    wine.loc[wine[col] < 0, col] = median_val

wine = wine.drop_duplicates()
for col in numeric_cols:
    if wine[col].isnull().sum() > 0:
        # thay bằng median
        wine[col].fillna(wine[col].median(), inplace=True)

print("\nKiểm tra dữ liệu thiếu sau khi xử lý:")
print(wine.isnull().sum())
print("\nSau khi xóa trùng lặp:")
print("Số dòng còn lại:", wine.shape[0])


Kiểm tra dữ liệu thiếu sau khi xử lý:
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

Sau khi xóa trùng lặp:
Số dòng còn lại: 1359


## 6. Bổ sung độ lệch và độ nhọn

In [57]:
from scipy.stats import skew, kurtosis
desc["skewness"] = wine.skew()
desc["kurtosis"] = wine.kurtosis()
print(desc)

                       count       mean        std      min      25%  \
fixed acidity         1599.0   8.319637   1.741096  4.60000   7.1000   
volatile acidity      1599.0   0.527821   0.179060  0.12000   0.3900   
citric acid           1599.0   0.270976   0.194801  0.00000   0.0900   
residual sugar        1599.0   2.538806   1.409928  0.90000   1.9000   
chlorides             1599.0   0.087467   0.047065  0.01200   0.0700   
free sulfur dioxide   1599.0  15.874922  10.460157  1.00000   7.0000   
total sulfur dioxide  1599.0  46.467792  32.895324  6.00000  22.0000   
density               1599.0   0.996747   0.001887  0.99007   0.9956   
pH                    1599.0   3.311113   0.154386  2.74000   3.2100   
sulphates             1599.0   0.658149   0.169507  0.33000   0.5500   
alcohol               1599.0  10.422983   1.065668  8.40000   9.5000   
quality               1599.0   5.636023   0.807569  3.00000   5.0000   

                           50%        75%        max  skewness 

**Nhận xét**:

Bổ sung độ lệch (skewness) và độ nhọn (kurtosis) giúp hiểu sâu hơn về hình dạng phân phối dữ liệu.
Một số biến như residual sugar và chlorides có độ lệch dương lớn → phân phối lệch phải mạnh, chứa nhiều giá trị ngoại lai.
Các biến có kurtosis cao (>3) cần được chuẩn hóa hoặc log-transform để giảm ảnh hưởng của outlier.

## 7. Thống kê mô tả


In [58]:
stats_summary = {}

for col in numeric_cols:
    data = wine[col]

    mean_val = np.mean(data)
    median_val = np.median(data)
    mode_val = stats.mode(data, keepdims=True)[0][0]
    var_val = np.var(data, ddof=1)  # variance
    std_val = np.std(data, ddof=1)  # standard deviation
    range_val = np.max(data) - np.min(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1

    stats_summary[col] = {
        "Mean": mean_val,
        "Median": median_val,
        "Mode": mode_val,
        "Variance": var_val,
        "Std Dev": std_val,
        "Range": range_val,
        "Q1 (25%)": q1,
        "Q3 (75%)": q3,
        "IQR": iqr
    }

In [59]:
stats_df = pd.DataFrame(stats_summary).T
print("\nThống kê mô tả:")
display(stats_df)


Thống kê mô tả:


,Mean,Median,Mode,Variance,Std Dev,Range,Q1 (25%),Q3 (75%),IQR
fixed acidity,8.310596,7.9000,7.2000,3.017134,1.736990,11.30000,7.1000,9.20000,2.10000
volatile acidity,0.529478,0.5200,0.5000,0.033500,0.183031,1.46000,0.3900,0.64000,0.25000
citric acid,0.272333,0.2600,0.0000,0.038235,0.195537,1.00000,0.0900,0.43000,0.34000
residual sugar,2.523400,2.2000,2.0000,1.828752,1.352314,14.60000,1.9000,2.60000,0.70000
chlorides,0.088124,0.0790,0.0800,0.002438,0.049377,0.59900,0.0700,0.09100,0.02100
free sulfur dioxide,15.893304,14.0000,6.0000,109.145456,10.447270,71.00000,7.0000,21.00000,14.00000
total sulfur dioxide,46.825975,38.0000,28.0000,1116.157653,33.408946,283.00000,22.0000,63.00000,41.00000
density,0.996709,0.9967,0.9968,0.000003,0.001869,0.01362,0.9956,0.99782,0.00222
pH,3.309787,3.3100,3.3000,0.024036,0.155036,1.27000,3.2100,3.40000,0.19000
sulphates,0.658705,0.6200,0.5400,0.029127,0.170667,1.67000,0.5500,0.73000,0.18000


## 8. So sánh trung bình theo nhóm chất lượng

In [60]:
wine["quality_label"] = wine["quality"].apply(lambda q: "good" if q >= 6 else "bad")
print(wine["quality_label"].value_counts())
print(wine["quality_label"].value_counts(normalize=True))

quality_label
good    719
bad     640
Name: count, dtype: int64
quality_label
good    0.529065
bad     0.470935
Name: proportion, dtype: float64


In [61]:
groups = wine.groupby("quality_label")
for name, group in groups:
    print(f"==== {name} ====")
    print(group.describe().T)

==== bad ====
                      count       mean        std      min      25%  \
fixed acidity         640.0   8.141719   1.561787  4.60000   7.1000   
volatile acidity      640.0   0.592930   0.184137  0.18000   0.4600   
citric acid           640.0   0.237547   0.184963  0.00000   0.0800   
residual sugar        640.0   2.526797   1.339649  1.20000   1.9000   
chlorides             640.0   0.094144   0.058115  0.03900   0.0740   
free sulfur dioxide   640.0  16.659375  10.875864  3.00000   8.0000   
total sulfur dioxide  640.0  55.146094  36.996713  6.00000  23.7500   
density               640.0   0.997044   0.001596  0.99256   0.9961   
pH                    640.0   3.309016   0.154503  2.74000   3.2000   
sulphates             640.0   0.620484   0.181235  0.33000   0.5275   
alcohol               640.0   9.920781   0.773478  8.40000   9.4000   
quality               640.0   4.885937   0.364018  3.00000   5.0000   

                            50%       75%        max  
fixed a

**Nhận xét:**

Rượu “High” có alcohol cao hơn, volatile acidity thấp hơn rõ rệt so với nhóm “Low”. Đây là các yếu tố ảnh hưởng chính đến chất lượng cảm quan.

## 9. Thống kê mô tả chi tiết thủ công cho từng cột số

In [62]:
data = wine.select_dtypes(include=[np.number])

for col in data.columns:
    values = data[col].values

    mean = np.mean(values)
    median = np.median(values)
    mode = stats.mode(values, keepdims=True)   # mode trả về object
    var = np.var(values, ddof=1)               # sample variance (ddof=1)
    std = np.std(values, ddof=1)               # sample standard deviation
    data_range = np.max(values) - np.min(values)
    q1 = np.percentile(values, 25)
    q3 = np.percentile(values, 75)
    iqr = q3 - q1

    print(f"\n=== {col} ===")
    print(f"Mean: {mean:.4f}")
    print(f"Median: {median:.4f}")
    print(f"Mode: {mode.mode[0]} (Count={mode.count[0]})")
    print(f"Variance: {var:.4f}")
    print(f"Standard Deviation: {std:.4f}")
    print(f"Range: {data_range:.4f}")
    print(f"Q1 (25%): {q1:.4f}")
    print(f"Q3 (75%): {q3:.4f}")
    print(f"IQR: {iqr:.4f}")


=== fixed acidity ===
Mean: 8.3106
Median: 7.9000
Mode: 7.2 (Count=49)
Variance: 3.0171
Standard Deviation: 1.7370
Range: 11.3000
Q1 (25%): 7.1000
Q3 (75%): 9.2000
IQR: 2.1000

=== volatile acidity ===
Mean: 0.5295
Median: 0.5200
Mode: 0.5 (Count=37)
Variance: 0.0335
Standard Deviation: 0.1830
Range: 1.4600
Q1 (25%): 0.3900
Q3 (75%): 0.6400
IQR: 0.2500

=== citric acid ===
Mean: 0.2723
Median: 0.2600
Mode: 0.0 (Count=118)
Variance: 0.0382
Standard Deviation: 0.1955
Range: 1.0000
Q1 (25%): 0.0900
Q3 (75%): 0.4300
IQR: 0.3400

=== residual sugar ===
Mean: 2.5234
Median: 2.2000
Mode: 2.0 (Count=133)
Variance: 1.8288
Standard Deviation: 1.3523
Range: 14.6000
Q1 (25%): 1.9000
Q3 (75%): 2.6000
IQR: 0.7000

=== chlorides ===
Mean: 0.0881
Median: 0.0790
Mode: 0.08 (Count=50)
Variance: 0.0024
Standard Deviation: 0.0494
Range: 0.5990
Q1 (25%): 0.0700
Q3 (75%): 0.0910
IQR: 0.0210

=== free sulfur dioxide ===
Mean: 15.8933
Median: 14.0000
Mode: 6.0 (Count=121)
Variance: 109.1455
Standard Deviatio

**Nhận xét**:

+ Các biến có độ lệch cao như residual sugar, chlorides có phân bố không chuẩn.

+ Các đặc trưng như alcohol có Mean > Median, cho thấy xu hướng lệch phải nhẹ.

+ IQR nhỏ ở pH → dữ liệu tập trung quanh trung bình, ít biến động.

+ Có thể chuẩn hóa hoặc log-transform một số biến để giảm ảnh hưởng outlier.

# Bài tập thực hành 2: Thống kê mô tả dữ liệu tiểu đường Pima

**Mục tiêu:**  
Thực hiện thống kê mô tả trên tập dữ liệu **Pima Indians Diabetes Dataset** — dữ liệu y tế dùng để dự đoán khả năng mắc bệnh tiểu đường.  
Nguồn dữ liệu: [Pima Indians Diabetes EDA & Prediction (Kaggle)](https://www.kaggle.com/code/vincentlugat/pima-indians-diabetes-eda-prediction-0-906).  

**Nội dung thực hiện:**  
- Kiểm tra, làm sạch và mô tả dữ liệu (mean, median, mode, min, max,...).  
- Quan sát phân phối của các biến như Glucose, BMI, Age, Insulin,...

**Mục đích:**  
Xác định các yếu tố chính ảnh hưởng đến nguy cơ mắc bệnh tiểu đường, và chuẩn bị nền tảng cho các mô hình học máy (logistic regression, decision tree, v.v.).


Nhập các thư viện cần thiết cho phân tích dữ liệu (thường là pandas, numpy, matplotlib, seaborn, sklearn,...). Các thư viện này hỗ trợ đọc/ghi dữ liệu, xử lý số liệu, và trực quan hóa.

## 1. Nhập thư viện

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

## 2. Đọc dữ liệu và Thông tin tổng quan



In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/EDA/diabetes.csv")
# shape
print(f'+ Shape: {df.shape}')
# types
print(f'+ Data Types: \n{df.dtypes}')
# head, tail
print(f'+ Contents: ')
display(df.head(5))
display(df.tail(5))
# info
df.info()
desc = df.describe().T

+ Shape: (768, 9)
+ Data Types: 
Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object
+ Contents: 


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


### Nhận xét kết quả

Nhận xét: Sau khi đọc dữ liệu, cần kiểm tra kích thước (shape), kiểu dữ liệu (dtypes) và một vài dòng đầu để đảm bảo dữ liệu được nạp đúng. Nếu file không tồn tại hoặc đường dẫn sai sẽ gây lỗi FileNotFound.

## 3. Kiểm tra dữ liệu thiếu và Loại bỏ dữ liệu trùng lặp


In [ ]:
has_null = df.isnull().sum().any()
has_nan  = df.isna().sum().any()
n_duplicated = df.duplicated().sum()

print(f'Tính toàn vẹn dữ liệu:')
print(f'+ Có giá trị Null: {has_null}')
if has_null:
    print(df[df.isnull().any(axis=1)])
print(f'+ Có giá trị Nan: {has_nan}')
if has_nan:
    print(df[df.isna().any(axis=1)])
print(f'+ Số dòng trùng: {n_duplicated}')

# Kiểm tra số lượng giá trị 0 ở các cột mà 0 là vô lý
print('Số lượng các giá trị 0 ở các cột:')
cols_zero_invalid = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
print((df[cols_zero_invalid] == 0).sum())

Tính toàn vẹn dữ liệu:
+ Có giá trị Null: False
+ Có giá trị Nan: False
+ Số dòng trùng: 0
Số lượng các giá trị 0 ở các cột:
Glucose            5
BloodPressure     35
SkinThickness    227
Insulin          374
BMI               11
dtype: int64


### Nhận xét kết quả

Kiểm tra kỹ kết quả cell để đảm bảo logic chuỗi xử lý và không gây lỗi downstream. Thông tin này cho biết biến nào là numeric/text và có bao nhiêu giá trị thiếu, từ đó tác động đến chiến lược xử lý dữ liệu tiếp theo.

## 4. Xử lý dữ liệu thiếu
Thay thế NaN bằng median


In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    median_val = df[col].median()
    df.loc[df[col] < 0, col] = median_val

for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        # thay bằng median
        df[col].fillna(df[col].median(), inplace=True)

print("\nKiểm tra dữ liệu thiếu sau khi xử lý:")
print(df.isnull().sum())


Kiểm tra dữ liệu thiếu sau khi xử lý:
Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


/tmp/ipython-input-1391406571.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '30.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df[col] < 0, col] = median_val


### Nhận xét kết quả

Xác định biến nào cần xử lý missing. Tỷ lệ missing lớn ở một số biến quyết định phương án: loại dòng, điền giá trị trung vị/trung bình, hoặc sử dụng mô hình imputation.

## 5. Bổ sung độ lệch và độ nhọn

In [ ]:
from scipy.stats import skew, kurtosis
desc["skewness"] = df.skew()
desc["kurtosis"] = df.kurtosis()
print(desc)

                          count        mean         std     min       25%  \
Pregnancies               768.0    3.845052    3.369578   0.000   1.00000   
Glucose                   768.0  120.894531   31.972618   0.000  99.00000   
BloodPressure             768.0   69.105469   19.355807   0.000  62.00000   
SkinThickness             768.0   20.536458   15.952218   0.000   0.00000   
Insulin                   768.0   79.799479  115.244002   0.000   0.00000   
BMI                       768.0   31.992578    7.884160   0.000  27.30000   
DiabetesPedigreeFunction  768.0    0.471876    0.331329   0.078   0.24375   
Age                       768.0   33.240885   11.760232  21.000  24.00000   
Outcome                   768.0    0.348958    0.476951   0.000   0.00000   

                               50%        75%     max  skewness  kurtosis  
Pregnancies                 3.0000    6.00000   17.00  0.901674  0.159220  
Glucose                   117.0000  140.25000  199.00  0.173754  0.640780  
B

## 6. Thống kê mô tả


In [ ]:
stats_summary = {}

for col in numeric_cols:
    data = df[col]

    mean_val = np.mean(data)
    median_val = np.median(data)
    mode_val = stats.mode(data, keepdims=True)[0][0]
    var_val = np.var(data, ddof=1)  # variance
    std_val = np.std(data, ddof=1)  # standard deviation
    range_val = np.max(data) - np.min(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1

    stats_summary[col] = {
        "Mean": mean_val,
        "Median": median_val,
        "Mode": mode_val,
        "Variance": var_val,
        "Std Dev": std_val,
        "Range": range_val,
        "Q1 (25%)": q1,
        "Q3 (75%)": q3,
        "IQR": iqr
    }

In [ ]:
stats_df = pd.DataFrame(stats_summary).T
print("\nThống kê mô tả:")
display(stats_df)


Thống kê mô tả:


,Mean,Median,Mode,Variance,Std Dev,Range,Q1 (25%),Q3 (75%),IQR
Pregnancies,3.845052,3.0000,1.000,11.354056,3.369578,17.000,1.00000,6.00000,5.0000
Glucose,120.894531,117.0000,99.000,1022.248314,31.972618,199.000,99.00000,140.25000,41.2500
BloodPressure,69.105469,72.0000,70.000,374.647271,19.355807,122.000,62.00000,80.00000,18.0000
SkinThickness,20.536458,23.0000,0.000,254.473245,15.952218,99.000,0.00000,32.00000,32.0000
Insulin,79.799479,30.5000,0.000,13281.180078,115.244002,846.000,0.00000,127.25000,127.2500
BMI,31.992578,32.0000,32.000,62.159984,7.884160,67.100,27.30000,36.60000,9.3000
DiabetesPedigreeFunction,0.471876,0.3725,0.254,0.109779,0.331329,2.342,0.24375,0.62625,0.3825
Age,33.240885,29.0000,22.000,138.303046,11.760232,60.000,24.00000,41.00000,17.0000
Outcome,0.348958,0.0000,0.000,0.227483,0.476951,1.000,0.00000,1.00000,1.0000


## 7. So sánh trung bình các đặc trưng theo nhóm Outcome

In [ ]:
if "Outcome" in df.columns:
    print("\nSo sánh trung bình các đặc trưng theo nhóm Outcome:")
    display(df.groupby("Outcome").mean(numeric_only=True))
else:
    print("\nKhông tìm thấy cột 'Outcome' trong dữ liệu.")


So sánh trung bình các đặc trưng theo nhóm Outcome:


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


**Nhận xét:**

Các đặc trưng Glucose, BMI, Insulin, Age và DiabetesPedigreeFunction có sự khác biệt rõ ràng giữa hai nhóm Outcome.
→ Đây là những biến quan trọng cần chú ý trong phân tích, mô hình dự đoán hoặc kiểm định giả thuyết.

Ngược lại, BloodPressure và SkinThickness có mức chênh lệch ít hơn, nên ít ảnh hưởng hơn trong việc phân loại kết quả bệnh.

## 8. Thống kê mô tả chi tiết thủ công cho từng cột số

In [ ]:
data = df.select_dtypes(include=[np.number])

for col in data.columns:
    values = data[col].dropna().values  # loại bỏ giá trị thiếu nếu có

    mean = np.mean(values)
    median = np.median(values)
    mode = stats.mode(values, keepdims=True)
    var = np.var(values, ddof=1)
    std = np.std(values, ddof=1)
    data_range = np.max(values) - np.min(values)
    q1 = np.percentile(values, 25)
    q3 = np.percentile(values, 75)
    iqr = q3 - q1
    skew = stats.skew(values)
    kurt = stats.kurtosis(values)

    print(f"\n=== {col} ===")
    print(f"Count: {len(values)}")
    print(f"Mean: {mean:.4f}")
    print(f"Median: {median:.4f}")
    print(f"Mode: {mode.mode[0]} (Count={mode.count[0]})")
    print(f"Variance: {var:.4f}")
    print(f"Standard Deviation: {std:.4f}")
    print(f"Range: {data_range:.4f}")
    print(f"Q1 (25%): {q1:.4f}")
    print(f"Q3 (75%): {q3:.4f}")
    print(f"IQR: {iqr:.4f}")
    print(f"Skewness: {skew:.4f}")
    print(f"Kurtosis: {kurt:.4f}")


=== Pregnancies ===
Count: 768
Mean: 3.8451
Median: 3.0000
Mode: 1 (Count=135)
Variance: 11.3541
Standard Deviation: 3.3696
Range: 17.0000
Q1 (25%): 1.0000
Q3 (75%): 6.0000
IQR: 5.0000
Skewness: 0.8999
Kurtosis: 0.1504

=== Glucose ===
Count: 768
Mean: 120.8945
Median: 117.0000
Mode: 99 (Count=17)
Variance: 1022.2483
Standard Deviation: 31.9726
Range: 199.0000
Q1 (25%): 99.0000
Q3 (75%): 140.2500
IQR: 41.2500
Skewness: 0.1734
Kurtosis: 0.6288

=== BloodPressure ===
Count: 768
Mean: 69.1055
Median: 72.0000
Mode: 70 (Count=57)
Variance: 374.6473
Standard Deviation: 19.3558
Range: 122.0000
Q1 (25%): 62.0000
Q3 (75%): 80.0000
IQR: 18.0000
Skewness: -1.8400
Kurtosis: 5.1387

=== SkinThickness ===
Count: 768
Mean: 20.5365
Median: 23.0000
Mode: 0 (Count=227)
Variance: 254.4732
Standard Deviation: 15.9522
Range: 99.0000
Q1 (25%): 0.0000
Q3 (75%): 32.0000
IQR: 32.0000
Skewness: 0.1092
Kurtosis: -0.5245

=== Insulin ===
Count: 768
Mean: 79.7995
Median: 30.5000
Mode: 0.0 (Count=374)
Variance: 13

### Nhận xét và phân tích kết quả

Kết quả thống kê mô tả cho thấy:  
- **Glucose** và **BMI** là hai yếu tố có mối tương quan cao nhất với khả năng mắc bệnh.  
- Nhóm mắc bệnh có xu hướng có **mức glucose và BMI cao hơn rõ rệt**.  
- Dữ liệu có một số giá trị 0 bất hợp lý ở các cột như `Insulin`, `BloodPressure`, cần được xử lý hoặc thay thế.  
- Phân phối tuổi cho thấy nguy cơ tăng dần theo độ tuổi.  

Tổng thể, tập dữ liệu phản ánh tốt các đặc điểm y học liên quan đến bệnh tiểu đường và rất thích hợp cho việc mô hình hóa dự đoán.  
